In [ ]:
# ! pip install flowcept[mlflow]

### Set the env var pointing to the conf file where the ports, hostnames, and other conf variables are read from.

There is an exemplary conf file available in the `resources` directory in FlowCept repository. You can use it as is if running this Notebook on your local laptop.

In [1]:
SQLITE_PATH = 'mlflow.db'

In [2]:
## This cell Resets MLFlow Database
! rm -f {SQLITE_PATH}
! rm -rf mlruns
import mlflow
mlflow.set_tracking_uri(f"sqlite:///{SQLITE_PATH}")
mlflow.delete_experiment(mlflow.create_experiment('starter'))

2024/09/27 10:59:42 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/09/27 10:59:42 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [3]:
# Sleeps are used because these notebooks are being tested automatically as part of the CI/CD. 
# In a normal user interaction, these sleeps would not be necessary.
from time import sleep

## Define a simple MLFlow workflow

In [4]:
def run_mlflow_workflow(sqlite_path=SQLITE_PATH, batch_size=64, epochs=10):
    import uuid
    import mlflow

    mlflow.set_tracking_uri(
        f"sqlite:///" f"{sqlite_path}"
    )
    experiment_name = "LinearRegression"
    experiment_id = mlflow.create_experiment(
        experiment_name + str(uuid.uuid4())
    )
    with mlflow.start_run(experiment_id=experiment_id) as run:
        mlflow.log_params({"epochs": epochs})
        mlflow.log_params({"batch_size": batch_size})
        # Actual training code would go here
        print("Generated training metadata.")
        mlflow.log_metric("loss", 0.04)
        return run.info.run_uuid

## Initialize interceptor

In [5]:
from flowcept import MLFlowInterceptor
interceptor = MLFlowInterceptor()

[flowcept][DEBUG][MAC132633][pid=74491][thread=8670187328][function=_build_logger][flowcept's base log is set up!]
DEBUG [flowcept] flowcept's base log is set up!


In [6]:
assert interceptor.settings.file_path == SQLITE_PATH

## Initialize consumer API

In [7]:
from flowcept import Flowcept
flowcept = Flowcept(interceptor)
flowcept.start()

## Run MLFlow workflow

In [12]:
mlflow_run_id = run_mlflow_workflow(interceptor.settings.file_path, batch_size=18)
print(f"MLflow task id={mlflow_run_id}")

Generated training metadata.
MLflow task id=e78450bf8e274e649e92b12f0e6a3fd8


In [ ]:
sleep(10)

## Initialize Query API

In [ ]:
from flowcept import TaskQueryAPI
query_api = TaskQueryAPI()

## Query the task executed

In [ ]:
_filter = {"task_id": mlflow_run_id}
query_api.query(_filter)

# Inspect more tasks through the query api

In [ ]:
from flowcept.commons.utils import get_utc_minutes_ago

### Get the tasks executed in my experiment in the last 60 minutes

This example assumes that you have run the Dask notebook example before. If you haven't run it, just ignore these queries.

In [ ]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "campaign_id": "super_campaign"
}
query_api.query(_filter)

### Get the tasks executed in my experiment in the last 60 minutes that were executed using mlflow and dask

In [ ]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "campaign_id": "super_campaign",
    "adapter_id": {"$in": ['mlflow', 'dask']}
}
docs = query_api.query(_filter)
docs

### Get the tasks executed in my experiment in the last 60 minutes that generated a batch_size > 0

In [ ]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(120) },
    "campaign_id": "super_campaign",
    "adapter_id": 'dask',
    "generated.batch_size": { "$gte" : 0 }
}
docs = query_api.query(filter=_filter)
batch_sizes = []
for doc in docs:
    print(f"task={doc['task_id']}, generated batch_size={doc['generated']['batch_size']}")
    batch_sizes.append(doc['generated']['batch_size'])
batch_sizes

### Now run a new MLFlow task using the batch_sizes generated by the Dask workflow

In [ ]:
batch_sizes = batch_sizes if len(batch_sizes) else [32]  # To use this if you haven't executed the Dask workflow first

In [ ]:
for batch_size in batch_sizes:
    mlflow_task = run_mlflow_workflow(batch_size=batch_size)
    print(mlflow_task)

In [ ]:
sleep(15)

### Get these tasks

In [ ]:
_filter = {
    "task_id": mlflow_task
}
docs = query_api.query(filter=_filter)
docs

In [ ]:
assert len(docs)

## Stop consumers

In [ ]:
flowcept.stop()